<a href="https://colab.research.google.com/github/LiscaAngriani/APM_Kelompok3/blob/main/Rule_Based_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [21]:
#reading data
df = pd.read_csv("persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


Variables (Features)


*   PRICE: Customer spend amount
*   SOURCE: The type of device the customer is connecting to (IOS/Android)
*   SEX: Gender of the customer
*   COUNTRY: Country of the customer
*   AGE: Age of the customer

In [22]:
# CHECK DF
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(5000, 5)
##################### Types #####################
PRICE       int64
SOURCE     object
SEX        object
COUNTRY    object
AGE         int64
dtype: object
##################### Head #####################
   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17
##################### Tail #####################
      PRICE   SOURCE     SEX COUNTRY  AGE
4995     29  android  female     bra   31
4996     29  android  female     bra   31
4997     29  android  female     bra   31
4998     39  android  female     bra   31
4999     29  android  female     bra   31
##################### NA #####################
PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64
##################### Quantiles #####################
       0.00  0.05  0.50  0.95  0.99

In [23]:
# UNIQUE SOURCES AND FREQUENCIES
sources_and_freq = pd.DataFrame({"SOURCES" : df["SOURCE"].unique() , "FREQUENCIES" : df["SOURCE"].value_counts()})
sources_and_freq

,SOURCES,FREQUENCIES
android,android,2974
ios,ios,2026


In [43]:
# UNIQUE PRICES
df["PRICE"].unique()

array([39, 49, 29, 19, 59,  9])

In [25]:
# HOW MANY SALES MADE FROM WHICH PRICE
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [26]:
# HOW MANY SALES MADE FROM WHICH COUNTRY
df["COUNTRY"].value_counts()

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [27]:
# HOW MUCH MONEY WAS MADE FROM EACH COUNTRY
df["PRICE"].groupby(df["COUNTRY"]).sum()
df.groupby("COUNTRY").agg({"PRICE" : "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [28]:
# HOW MUCH MONEY WAS MADE BY SOURCES
df.groupby("SOURCE").agg({"PRICE" : "sum"})

,PRICE
SOURCE,
android,101636
ios,69024


In [29]:
# MEAN PRICES FOR EACH COUNTRY
df.groupby("COUNTRY").agg({"PRICE" : "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [30]:
# MEAN PRICES FOR EACH COUNTRY AND SOURCE
df.groupby(["SOURCE","COUNTRY"]).agg({"PRICE" : "mean"})

PRICE
SOURCE  COUNTRY           
android bra      34.387029
        can      33.330709
        deu      33.869888
        fra      34.312500
        tur      36.229437
        usa      33.760357
ios     bra      34.222222
        can      33.951456
        deu      34.268817
        fra      32.776224
        tur      33.272727
        usa      34.371703

In [31]:
# MEAN PRICES FOR COUNTRY, SOURCE, AGE, SEX
df.groupby(["COUNTRY","SOURCE","AGE","SEX"]).agg({"PRICE" : "mean"})

PRICE
COUNTRY SOURCE  AGE SEX              
bra     android 15  female  38.714286
                    male    39.344828
                16  female  35.944444
                    male    29.666667
                17  female  35.666667
...                               ...
usa     ios     53  male    34.000000
                55  male    29.000000
                59  female  41.500000
                    male    46.500000
                65  female  35.000000

[348 rows x 1 columns]

In [32]:
# SORT VALUES BY PRICE
agg_df = df.groupby(["COUNTRY","SOURCE","AGE","SEX"]).agg({"PRICE" : "mean"}).sort_values(by = "PRICE", ascending=True)
agg_df.head()

PRICE
COUNTRY SOURCE  AGE SEX          
deu     android 26  male      9.0
tur     android 21  male     19.0
bra     ios     34  female   19.0
                47  male     19.0
can     android 27  female   19.0

In [33]:
# CONVERT INDICES TO COLUMNS
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,AGE,SEX,PRICE
0,deu,android,26,male,9.0
1,tur,android,21,male,19.0
2,bra,ios,34,female,19.0
3,bra,ios,47,male,19.0
4,can,android,27,female,19.0


In [34]:
# SET AGE AS CATEGORICAL AND SAVE IT AS NEW COLUMN
bins = [0, 19, 24, 31, 41, agg_df["AGE"].max()]

# SET LABELS
mylabels = ['0_18', '19_24', '24_30', '30_40', '40_' + str(agg_df["AGE"].max())]
agg_df["CAT_AGE"] = pd.cut(df["AGE"], bins , labels = mylabels)
agg_df.head()

,COUNTRY,SOURCE,AGE,SEX,PRICE,CAT_AGE
0,deu,android,26,male,9.0,0_18
1,tur,android,21,male,19.0,0_18
2,bra,ios,34,female,19.0,0_18
3,bra,ios,47,male,19.0,0_18
4,can,android,27,female,19.0,0_18


In [35]:
# CREATE CUSTOMER LEVEL BASED COLUMN
agg_df["CUSTOMER_LEVEL_BASED"] = [row[0].upper() + "_" + row[1].upper() + "_" + row[3].upper() + "_" + row[-1].upper() for row in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,AGE,SEX,PRICE,CAT_AGE,CUSTOMER_LEVEL_BASED
0,deu,android,26,male,9.0,0_18,DEU_ANDROID_MALE_0_18
1,tur,android,21,male,19.0,0_18,TUR_ANDROID_MALE_0_18
2,bra,ios,34,female,19.0,0_18,BRA_IOS_FEMALE_0_18
3,bra,ios,47,male,19.0,0_18,BRA_IOS_MALE_0_18
4,can,android,27,female,19.0,0_18,CAN_ANDROID_FEMALE_0_18


In [36]:
# REMOVE UNNECESSARY COLUMNS
agg_df = agg_df[["CUSTOMER_LEVEL_BASED","PRICE"]]
agg_df.head()

,CUSTOMER_LEVEL_BASED,PRICE
0,DEU_ANDROID_MALE_0_18,9.0
1,TUR_ANDROID_MALE_0_18,19.0
2,BRA_IOS_FEMALE_0_18,19.0
3,BRA_IOS_MALE_0_18,19.0
4,CAN_ANDROID_FEMALE_0_18,19.0


In [37]:
# GROUP BY LEVELS
agg_df = agg_df.groupby("CUSTOMER_LEVEL_BASED").agg({"PRICE" : "mean"})

agg_df = agg_df.reset_index()

agg_df.head()

,CUSTOMER_LEVEL_BASED,PRICE
0,BRA_ANDROID_FEMALE_0_18,34.005576
1,BRA_ANDROID_FEMALE_19_24,34.619389
2,BRA_ANDROID_FEMALE_24_30,40.382114
3,BRA_ANDROID_FEMALE_30_40,35.713503
4,BRA_ANDROID_FEMALE_40_66,32.333333


In [38]:
# SEGMENTATION BY PRICE
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, ["D","C","B","A"])
agg_df.head()

,CUSTOMER_LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,34.005576,C
1,BRA_ANDROID_FEMALE_19_24,34.619389,B
2,BRA_ANDROID_FEMALE_24_30,40.382114,A
3,BRA_ANDROID_FEMALE_30_40,35.713503,B
4,BRA_ANDROID_FEMALE_40_66,32.333333,C


In [45]:
# SOME STATISTICS ABOUT SEGMENTS
agg_df.groupby("SEGMENT").agg({"PRICE" : ["min","max","mean","sum"]})

PRICE                                  
               min        max       mean         sum
SEGMENT                                             
D        25.000000  31.632076  29.482609  737.065214
C        31.724328  34.005576  32.967427  791.218255
B        34.018141  36.140152  35.133631  843.207134
A        36.150455  45.000000  38.459939  961.498480

In [40]:
# WHICH SEGMENT DOES A 33 YEARS OLD TURKISH WOMAN USING ANDROID BELONG TO ?
new_user = "TUR_ANDROID_FEMALE_30_40"
agg_df[agg_df["CUSTOMER_LEVEL_BASED"] == new_user]

,CUSTOMER_LEVEL_BASED,PRICE,SEGMENT
67,TUR_ANDROID_FEMALE_30_40,33.010417,C


In [41]:
# WHICH SEGMENT DOES A 35 YEARS OLD FRENCH WOMAN USING IOS BELONG TO ?
new_user = "FRA_IOS_FEMALE_30_40"
agg_df[agg_df["CUSTOMER_LEVEL_BASED"] == new_user]

,CUSTOMER_LEVEL_BASED,PRICE,SEGMENT
59,FRA_IOS_FEMALE_30_40,33.093407,C
